<a href="https://colab.research.google.com/github/yehoon17/BERT_sandwich_order/blob/main/data_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import random

keys = ['sandwich','length','cheese','bread','vegetable','sauce']
kor_keys = ['샌드위치','길이','치즈','빵','채소','소스']

for key, kor_key in zip(keys,kor_keys):
    exec("%s = tuple(pd.read_csv('%s.csv'))"%(key,kor_key))
    print('<'+kor_key+'>')
    exec("print(*%s,sep=', ')"%(key))
    print()

<샌드위치>
페퍼 치킨 슈니첼, 페퍼로니 피자 썹, 쉬림프, 로스트 치킨, 에그마요, K-바비큐, 로티세리 바비큐 치킨, 풀드 포크 바비큐, 이탈리안 비엠티, 이탈리안 BLT, BMT, 비엘티, 미트볼, 햄, 참치, 써브웨이 클럽, 터키, 베지, 스테이크 & 치즈, 스테이크 앤 치즈, 터키 베이컨 아보카도, 스파이시 이탈리안, 치킨 데리야끼

<길이>
15센치, 15센치미터, 15센티, 15센티미터, 6인치, 6inch, 15cm, 30cm, 30센치, 30센티, 30센치미터, 30센티미터, 풋롱

<치즈>
아메리칸, 슈레드, 모차렐라, 모짜렐라, 아메리칸 치즈, 슈레드 치츠, 모차렐라 치츠

<빵>
화이트, 하티, 파마산오레가노, 위트, 허니오트, 플랫, 파마산

<채소>
양상추, 토마토, 오이, 피망, 양파, 피클, 올리브, 할라피뇨

<소스>
랜치, 마요네즈, 스위트 어니언, 허니 머스타드, 스위트 칠리, 핫 칠리, 사우스웨스트 치폴레, 머스타드, 홀스래디쉬, 올리브 오일, 레드와인식초, 소금, 후추, 스모크 바비큐



In [14]:
menu = {}
for key in keys:
    exec("menu['%s'] = %s"%(key,key))

In [15]:
sample_text = '15센치 화이트 빵으로 비엘티에 아메리칸 치즈 올려서\
 피망하고 파프리카 빼고 랜치랑 스위트 칠리 뿌려서 주세요..'

In [16]:
sample_data = '/length;15센치/ /bread;화이트/ 빵으로 /sandwich;비엘티/에\
 /cheese;아메리칸 치즈/ 올려서 /vegetable;피망/하고 /vegetable;파프리카/ 빼고\
 /sauce;랜치/랑 /sauce;스위트 칠리/ 뿌려서 주세요.'

In [17]:
def has_coda(word):
    # 받침으로 끝나면 false, 아니면 true 리턴
    if is_hangul(word):
        return (ord(word[-1]) - 44032) % 28 == 0
    return True

def is_hangul(word):
    code = ord(word[-1])
    if 44032 <= code <= 55203:
        return True
    return False

In [18]:
cat = {key:[val] for key,val in zip(keys, kor_keys)}
cat['length']+=['크기','사이즈']
cat['vegetable'].append('야채')
cat

{'sandwich': ['샌드위치'],
 'length': ['길이', '크기', '사이즈'],
 'cheese': ['치즈'],
 'bread': ['빵'],
 'vegetable': ['채소', '야채'],
 'sauce': ['소스']}

In [19]:
data_with_one = []
pos_predicates = ['으로 해주세요.','으로 할게요.','으로 주세요.','으로 주이소.']
neg_predicate = ' 빼주세요.'
predicate ='이요'

for key,li in menu.items():
    for val in li:
        if key == 'vegetable':
            # "{채소} 빼주세요."
            data_with_one.append('/%s;%s/'%(key,val)+neg_predicate)
            # "{veg}는 {채소} 빼주세요."
            for veg in cat[key]:
                data_with_one.append(veg+'는'+'/%s;%s/'%(key,val)+neg_predicate)
            # "{채소}은(는) 빼주세요."
            if has_coda(val):
                data_with_one.append('/%s;%s/'%(key,val)+'는'+neg_predicate)
            else:
                data_with_one.append('/%s;%s/'%(key,val)+'은'+neg_predicate)
        else:
            for pos_predicate in pos_predicates:
                if has_coda(val):
                    pos_predicate = pos_predicate[1:]
                # "{무엇}(으)로 해주세요/할게요/주세요."
                data_with_one.append('/%s;%s/'%(key,val)+pos_predicate)
                # "{종류}은(는) {무엇}(으)로 해주세요/할게요/주세요."
                for x in cat[key]:
                    if has_coda(x):
                        data_with_one.append(x+'는 '+'/%s;%s/'%(key,val)+pos_predicate)
                    else:
                        data_with_one.append(x+'은 '+'/%s;%s/'%(key,val)+pos_predicate)
        temp = predicate
        if has_coda(val):
            temp = predicate[1:]
        # "{무엇}(이)요"
        data_with_one.append('/%s;%s/'%(key,val)+temp)
        
print(len(data_with_one))
print(*data_with_one[:10], sep='\n')

720
/sandwich;페퍼 치킨 슈니첼/으로 해주세요.
샌드위치는 /sandwich;페퍼 치킨 슈니첼/으로 해주세요.
/sandwich;페퍼 치킨 슈니첼/으로 할게요.
샌드위치는 /sandwich;페퍼 치킨 슈니첼/으로 할게요.
/sandwich;페퍼 치킨 슈니첼/으로 주세요.
샌드위치는 /sandwich;페퍼 치킨 슈니첼/으로 주세요.
/sandwich;페퍼 치킨 슈니첼/으로 주이소.
샌드위치는 /sandwich;페퍼 치킨 슈니첼/으로 주이소.
/sandwich;페퍼 치킨 슈니첼/이요
/sandwich;페퍼로니 피자 썹/으로 해주세요.


In [21]:
with open('data.csv', 'a') as f:
    for sentence in random.sample(data_with_one,200):
        f.write(sentence+'\n')

In [23]:
pre_oven_data1 = []
## 빵과 메뉴
# 파마산에 햄으로 주세요/할게요.
# 에그마요에 (빵은) 플렛으로 주세요.
pre_oven_bread_sandwich = []
for bread in menu['bread']:
    for sandwich in menu['sandwich']:
        predicate = '으로 주세요.'
        if has_coda(bread):
            predicate = '로 주세요.'
        order = '/sandwich;'+sandwich+'/에 '
        pre_oven_data1.append(order+'/bread;'+bread+'/'+predicate)
        pre_oven_data1.append(order+'빵은 /bread;'+bread+'/'+predicate)
        
        predicate = '으로 주세요.'
        if has_coda(sandwich):
            predicate = '로 주세요.'
        order = '/bread;'+bread+'/에 '+'/sandwich;'+sandwich+'/'+predicate
        pre_oven_data1.append(order)

        pre_oven_bread_sandwich.append(order[:-5]+' 해주시고, ')
        
print(len(pre_oven_data1))
print(*pre_oven_data1[:20], sep='\n') 

483
/sandwich;페퍼 치킨 슈니첼/에 /bread;화이트/로 주세요.
/sandwich;페퍼 치킨 슈니첼/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 주세요.
/sandwich;페퍼로니 피자 썹/에 /bread;화이트/로 주세요.
/sandwich;페퍼로니 피자 썹/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;페퍼로니 피자 썹/으로 주세요.
/sandwich;쉬림프/에 /bread;화이트/로 주세요.
/sandwich;쉬림프/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;쉬림프/로 주세요.
/sandwich;로스트 치킨/에 /bread;화이트/로 주세요.
/sandwich;로스트 치킨/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;로스트 치킨/으로 주세요.
/sandwich;에그마요/에 /bread;화이트/로 주세요.
/sandwich;에그마요/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;에그마요/로 주세요.
/sandwich;K-바비큐/에 /bread;화이트/로 주세요.
/sandwich;K-바비큐/에 빵은 /bread;화이트/로 주세요.
/bread;화이트/에 /sandwich;K-바비큐/로 주세요.
/sandwich;로티세리 바비큐 치킨/에 /bread;화이트/로 주세요.
/sandwich;로티세리 바비큐 치킨/에 빵은 /bread;화이트/로 주세요.


In [24]:
with open('data.csv', 'a') as f:
    for sentence in random.sample(pre_oven_data1,200):
        f.write(sentence+'\n')

In [25]:
pre_oven_data2 = []
## 빵과 길이
# (빵은) 파마산 오레가노 15센치(요./로 (해)주세요.)
# (빵은) 15센치 파마산 오레가노로 해주세요

for bread in menu['bread']:
    for length in menu['length']:
        predicate = '으로 해주세요.'
        if has_coda(bread):
            predicate = '로 해주세요.'
        order = '/length;'+length+'/ '+'/bread;'+bread+'/'+predicate
        pre_oven_data2.append('빵은 '+order)
        pre_oven_data2.append(order)
        
        predicate = '으로 해주세요.'
        if has_coda(length):
            predicate = '로 해주세요.'
        order = '/bread;'+bread+'/ '+'/length;'+length+'/'+predicate
        pre_oven_data2.append('빵은 '+order)
        pre_oven_data2.append(order)
  
print(len(pre_oven_data2))
print(*pre_oven_data2[:20], sep='\n') 

364
빵은 /length;15센치/ /bread;화이트/로 해주세요.
/length;15센치/ /bread;화이트/로 해주세요.
빵은 /bread;화이트/ /length;15센치/로 해주세요.
/bread;화이트/ /length;15센치/로 해주세요.
빵은 /length;15센치미터/ /bread;화이트/로 해주세요.
/length;15센치미터/ /bread;화이트/로 해주세요.
빵은 /bread;화이트/ /length;15센치미터/로 해주세요.
/bread;화이트/ /length;15센치미터/로 해주세요.
빵은 /length;15센티/ /bread;화이트/로 해주세요.
/length;15센티/ /bread;화이트/로 해주세요.
빵은 /bread;화이트/ /length;15센티/로 해주세요.
/bread;화이트/ /length;15센티/로 해주세요.
빵은 /length;15센티미터/ /bread;화이트/로 해주세요.
/length;15센티미터/ /bread;화이트/로 해주세요.
빵은 /bread;화이트/ /length;15센티미터/로 해주세요.
/bread;화이트/ /length;15센티미터/로 해주세요.
빵은 /length;6인치/ /bread;화이트/로 해주세요.
/length;6인치/ /bread;화이트/로 해주세요.
빵은 /bread;화이트/ /length;6인치/로 해주세요.
/bread;화이트/ /length;6인치/로 해주세요.


In [26]:
with open('data.csv', 'a') as f:
    for sentence in random.sample(pre_oven_data2,200):
        f.write(sentence+'\n')

In [27]:
pre_oven_data3 = []         
## 빵, 길이, 메뉴
# 파마산 15샌치에 햄으로 주세요.

for bread in menu['bread']:
    bread = '/bread;'+bread+'/'
    for length in menu['length']:
        length = '/length;'+length+'/'
        bread_infos = [bread, bread+' '+length, length+' '+bread]
        for bread_info in bread_infos:
            for sandwich in menu['sandwich']:
                predicate = '으로 주세요.'
                if has_coda(sandwich):
                    predicate = '로 주세요.'
                sandwich = '/sandwich;'+sandwich+'/'
                ### {무엇}에(다가) {무엇}으로 주세요.
                # (빵)(+길이), (메뉴)
                pre_oven_data3.append(bread_info+'에다가 '+sandwich+predicate)
                ###빵은 {빵}으로 해주시고, {메뉴}로 주세요
                   

print(len(pre_oven_data3))
print(*pre_oven_data3[:20], sep='\n') 

6279
/bread;화이트/에다가 /sandwich;페퍼 치킨 슈니첼/으로 주세요.
/bread;화이트/에다가 /sandwich;페퍼로니 피자 썹/으로 주세요.
/bread;화이트/에다가 /sandwich;쉬림프/로 주세요.
/bread;화이트/에다가 /sandwich;로스트 치킨/으로 주세요.
/bread;화이트/에다가 /sandwich;에그마요/로 주세요.
/bread;화이트/에다가 /sandwich;K-바비큐/로 주세요.
/bread;화이트/에다가 /sandwich;로티세리 바비큐 치킨/으로 주세요.
/bread;화이트/에다가 /sandwich;풀드 포크 바비큐/로 주세요.
/bread;화이트/에다가 /sandwich;이탈리안 비엠티/로 주세요.
/bread;화이트/에다가 /sandwich;이탈리안 BLT/로 주세요.
/bread;화이트/에다가 /sandwich;BMT/로 주세요.
/bread;화이트/에다가 /sandwich;비엘티/로 주세요.
/bread;화이트/에다가 /sandwich;미트볼/으로 주세요.
/bread;화이트/에다가 /sandwich;햄/으로 주세요.
/bread;화이트/에다가 /sandwich;참치/로 주세요.
/bread;화이트/에다가 /sandwich;써브웨이 클럽/으로 주세요.
/bread;화이트/에다가 /sandwich;터키/로 주세요.
/bread;화이트/에다가 /sandwich;베지/로 주세요.
/bread;화이트/에다가 /sandwich;스테이크 & 치즈/로 주세요.
/bread;화이트/에다가 /sandwich;스테이크 앤 치즈/로 주세요.


In [28]:
with open('data.csv', 'a') as f:
    for sentence in random.sample(pre_oven_data3,200):
        f.write(sentence+'\n')

In [29]:
pre_oven_data4 = []  

## 빵, 메뉴, 치즈
# 화이트 빵에 이탈리안 비엠티 해주시고, 치즈는 아메리칸 치즈로 할게요.
for cheese in menu['cheese']:
    predicate = '으로 할게요.'
    if has_coda(cheese):
        predicate = '로 할게요.'
    order = '치즈는 '+'/cheese;'+cheese+'/'+predicate
    for x in pre_oven_bread_sandwich:
        pre_oven_data4.append(x+order)
  
print(len(pre_oven_data4))
print(*pre_oven_data4[:20], sep='\n')   

1127
/bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;페퍼로니 피자 썹/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;쉬림프/로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;로스트 치킨/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;에그마요/로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;K-바비큐/로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;로티세리 바비큐 치킨/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;풀드 포크 바비큐/로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;이탈리안 비엠티/로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;이탈리안 BLT/로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;BMT/로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;비엘티/로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;미트볼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;햄/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;참치/로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/bread;화이트/에 /sandwich;써브웨이 클럽/으로 해주시고, 치즈는 /chees

In [30]:
with open('data.csv', 'a') as f:
    for sentence in random.sample(pre_oven_data4,200):
        f.write(sentence+'\n')

In [31]:
pre_oven_data5 = []  

## 빵, 길이, 메뉴, 치즈
# 15cm 화이트에 이탈리안 비엠티 해주시고, 치즈는 아메리칸 치즈로 할게요.
for cheese in menu['cheese']:
    predicate = '으로 할게요.'
    if has_coda(cheese):
        predicate = '로 할게요.'
    order = '치즈는 '+'/cheese;'+cheese+'/'+predicate
    for x in pre_oven_bread_sandwich:
        for length in menu['length']:
            pre_oven_data5.append('/length;'+length+'/ '+x+order)


print(len(pre_oven_data5))
print(*pre_oven_data5[:20], sep='\n') 

14651
/length;15센치/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;15센치미터/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;15센티/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;15센티미터/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;6인치/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;6inch/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;15cm/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;30cm/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;30센치/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;30센티/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;30센치미터/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;30센티미터/ /bread;화이트/에 /sandwich;페퍼 치킨 슈니첼/으로 해주시고, 치즈는 /cheese;아메리칸/으로 할게요.
/length;풋롱/ /b

In [32]:
with open('data.csv', 'a') as f:
    for sentence in random.sample(pre_oven_data5,200):
        f.write(sentence+'\n')

In [33]:
from itertools import permutations

mult_veg_data = []
post_oven_veg = []
# 양파랑 피망 빼주세요.
# 채소/야채는 양파하고 피망 빼주세요.
# 양파랑 피망이랑 할라피뇨요.
# 양파, 피망, 할라피뇨요.
# 양파하고 피망하고 할라피뇨요.
# 양파 빼주시고 피망도 빼주세요.
# 양파, 피망 그리고 할라피뇨요.
connectives = ['이랑 ','하고 ',', ']
veges = menu['vegetable']
MAX_VEGE = 4
for i in range(2,MAX_VEGE+1):
    for p_veg in permutations(veges,i):
        order = ''
        for veg in p_veg:
            order+='/vegetable;'+veg+'/'+', '
        post_oven_veg.append(order[:-2] + ' 빼주시고, ')
        for p_con in permutations(connectives,i-1):
            order = ''
            for veg,con in zip(p_veg[:-1],p_con):
                order+='/vegetable;'+veg+'/'
                if con == '이랑 ':
                    if has_coda(veg):
                        con=con[1:]
                order+=con
            order+='/vegetable;'+p_veg[-1]+'/'
            # "{채소}(랑/하고/,) {채소}(랑/하고/,) {채소}요."
            order2=order
            if not has_coda(p_veg[-1]):
                order2+='이'
            mult_veg_data.append(order2+'요.')
            order+=' 빼주세요.'
            # "{채소}(랑/하고/,) {채소}(랑/하고/,) {채소} 빼주세요."
            mult_veg_data.append(order)
            for veg in cat['vegetable']:
                # "{veg}는 {채소}(랑/하고/,) {채소}(랑/하고/,) {채소} 빼주세요."
                mult_veg_data.append(veg+'는 '+order)
                
            order = ''
            for veg,con in zip(p_veg[:-2],p_con):
                order+='/vegetable;'+veg+'/'
                if con == '이랑 ':
                    if has_coda(veg):
                        con=con[1:]
                order+=con
            order+='/vegetable;'+p_veg[-2]+'/'+' 빼주시고 '
            order+='/vegetable;'+p_veg[-1]+'/'+'도 빼주세요.'
            # "{채소}(랑/하고/,) {채소} 빼주시고 {채소}도 빼주세요."
            mult_veg_data.append(order)
            for veg in cat['vegetable']:
                # "{veg}는 {채소}(랑/하고/,) {채소} 빼주시고 {채소}도 빼주세요."
                mult_veg_data.append(veg+'는 '+order)
                
        order=''
        for veg in p_veg[:-2]:
            order+='/vegetable;'+veg+'/'+', '
        order+='/vegetable;'+p_veg[-2]+'/'+' 그리고 '
        order+='/vegetable;'+p_veg[-1]+'/'+'요.'
        # "{채소}, {채소} 그리고 {채소}요."
        mult_veg_data.append(order)
            
print(len(mult_veg_data))
print(*mult_veg_data[:25], sep='\n')                    

87920
/vegetable;양상추/랑 /vegetable;토마토/요.
/vegetable;양상추/랑 /vegetable;토마토/ 빼주세요.
채소는 /vegetable;양상추/랑 /vegetable;토마토/ 빼주세요.
야채는 /vegetable;양상추/랑 /vegetable;토마토/ 빼주세요.
/vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
채소는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
야채는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
/vegetable;양상추/하고 /vegetable;토마토/요.
/vegetable;양상추/하고 /vegetable;토마토/ 빼주세요.
채소는 /vegetable;양상추/하고 /vegetable;토마토/ 빼주세요.
야채는 /vegetable;양상추/하고 /vegetable;토마토/ 빼주세요.
/vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
채소는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
야채는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
/vegetable;양상추/, /vegetable;토마토/요.
/vegetable;양상추/, /vegetable;토마토/ 빼주세요.
채소는 /vegetable;양상추/, /vegetable;토마토/ 빼주세요.
야채는 /vegetable;양상추/, /vegetable;토마토/ 빼주세요.
/vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
채소는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
야채는 /vegetable;양상추/ 빼주시고 /vegetable;토마토/도 빼주세요.
/vegetable;양상추/ 그리고 /vegetable;토마토/요.
/vegetable;양상추/랑 /vegetable;오이/요.
/vegetable;양상추/랑 /

In [34]:
with open('data.csv', 'a') as f:
    for sentence in random.sample(mult_veg_data,200):
        f.write(sentence+'\n')

In [36]:
mult_sauce_data = []
post_oven_sauce = []
connectives = ['이랑 ','하고 ',', ']
pos_predicates = ['으로 해주세요.','으로 할게요.','으로 주세요.','으로 주이소.']
sauces = menu['sauce']
MAX_SAUCE = 3
for i in range(2,MAX_SAUCE+1):
    for p_sauces in permutations(sauces,i):
        order = ''
        for sauce in p_sauces:
            order+='/sauce;'+sauce+'/'+', '
        post_oven_sauce.append(order[:-2] + ' 넣어주세요.')
        
        for p_con in permutations(connectives,i-1):
            order = ''
            for sauce,con in zip(p_sauces[:-1],p_con):
                order+='/sauce;'+sauce+'/'
                if con == '이랑 ':
                    if has_coda(sauce):
                        con=con[1:]
                order+=con
            order+='/sauce;'+p_sauces[-1]+'/'
            # "{소스}(랑/하고/,) {소스}(랑/하고/,) {소스}요."
            order2=order
            if not has_coda(p_sauces[-1]):
                order2+='이'
            mult_sauce_data.append(order2+'요.')
            
            for pos_predicate in pos_predicates:
                if has_coda(p_sauces[-1]):
                    pos_predicate = pos_predicate[1:]
                mult_sauce_data.append(order+pos_predicate)
                
print(len(mult_sauce_data))
print(*mult_sauce_data[:25], sep='\n')                    

68250
/sauce;랜치/랑 /sauce;마요네즈/요.
/sauce;랜치/랑 /sauce;마요네즈/로 해주세요.
/sauce;랜치/랑 /sauce;마요네즈/로 할게요.
/sauce;랜치/랑 /sauce;마요네즈/로 주세요.
/sauce;랜치/랑 /sauce;마요네즈/로 주이소.
/sauce;랜치/하고 /sauce;마요네즈/요.
/sauce;랜치/하고 /sauce;마요네즈/로 해주세요.
/sauce;랜치/하고 /sauce;마요네즈/로 할게요.
/sauce;랜치/하고 /sauce;마요네즈/로 주세요.
/sauce;랜치/하고 /sauce;마요네즈/로 주이소.
/sauce;랜치/, /sauce;마요네즈/요.
/sauce;랜치/, /sauce;마요네즈/로 해주세요.
/sauce;랜치/, /sauce;마요네즈/로 할게요.
/sauce;랜치/, /sauce;마요네즈/로 주세요.
/sauce;랜치/, /sauce;마요네즈/로 주이소.
/sauce;랜치/랑 /sauce;스위트 어니언/이요.
/sauce;랜치/랑 /sauce;스위트 어니언/으로 해주세요.
/sauce;랜치/랑 /sauce;스위트 어니언/으로 할게요.
/sauce;랜치/랑 /sauce;스위트 어니언/으로 주세요.
/sauce;랜치/랑 /sauce;스위트 어니언/으로 주이소.
/sauce;랜치/하고 /sauce;스위트 어니언/이요.
/sauce;랜치/하고 /sauce;스위트 어니언/으로 해주세요.
/sauce;랜치/하고 /sauce;스위트 어니언/으로 할게요.
/sauce;랜치/하고 /sauce;스위트 어니언/으로 주세요.
/sauce;랜치/하고 /sauce;스위트 어니언/으로 주이소.


In [37]:
with open('data.csv', 'a') as f:
    for sentence in random.sample(mult_sauce_data,200):
        f.write(sentence+'\n')

In [38]:
post_oven_data = []
for x in post_oven_veg:
    for y in post_oven_sauce:
        post_oven_data.append(x+y)
        
print(len(post_oven_data))
print(*post_oven_data[:25], sep='\n')                    

4902352
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;마요네즈/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;스위트 어니언/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;허니 머스타드/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;스위트 칠리/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;핫 칠리/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;사우스웨스트 치폴레/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;머스타드/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;홀스래디쉬/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;올리브 오일/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;레드와인식초/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;소금/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;후추/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;랜치/, /sauce;스모크 바비큐/ 넣어주세요.
/vegetable;양상추/, /vegetable;토마토/ 빼주시고, /sauce;마요네즈/

In [39]:
with open('data.csv', 'a') as f:
    for sentence in random.sample(post_oven_data,200):
        f.write(sentence+'\n')

In [42]:
from itertools import permutations

complex_data = []
## 완전 주문
# {빵} {길이} {샌드위치}에다가 치즈는 {치즈}로 해주시고, 채소는 {채소} 빼주시고,
# 소스는 {소스}로 해주세요.
complex_front = []
for bread in menu['bread']:
    bread = '/bread;'+bread+'/ '
    for length in menu['length']:
        length = '/length;'+length+'/ '
        for sandwich in menu['sandwich']:
            sandwich = '/sandwich;'+sandwich+'/'
            for cheese in menu['cheese']:
                cheese_predicate = '으로 해주시고, '
                if has_coda(cheese):
                    cheese_predicate = '로 해주시고, '
                cheese = '/cheese;'+cheese+'/'
                complex_front.append(bread+length+sandwich+\
                                    '에다가 치즈는 '+cheese+\
                                   cheese_predicate)
MAX_VEG = 1
MAX_SAUCE = 1
complex_back = []
for i in range(1,MAX_VEG+1):
    for p_veg in permutations(menu['vegetable'],i):
        vegs = ''
        for veg in p_veg:
            vegs+='/vegetable;'+veg+'/ '
        for i in range(1,MAX_SAUCE+1):
            for p_sauce in permutations(menu['sauce'],i):
                sauces = ''
                for sauce in p_sauce:
                    sauces+='/sauce;'+sauce+'/ '
                sauce_predicate = '으로 해주세요.'
                if has_coda(sauce):
                    sauce_predicate = '로 해주세요.'
                complex_back.append('채소는 '+\
                                   vegs+' 빼주시고, '+\
                                   sauces+sauce_predicate)

'''
print(len(complex_front)) 
print(len(complex_back))      
print(len(complex_back)*len(complex_front))

for front in complex_front:
    for back in complex_back:
      complex_data.append(front+back)
'''

for _ in range(200):
    complex_data.append(random.choice(complex_front)+random.choice(complex_back))

print(len(complex_data))
print(*complex_data[:25], sep='\n')   

200
/bread;하티/ /length;6inch/ /sandwich;이탈리안 BLT/에다가 치즈는 /cheese;모짜렐라/로 해주시고, 채소는 /vegetable;양파/  빼주시고, /sauce;후추/ 로 해주세요.
/bread;파마산오레가노/ /length;풋롱/ /sandwich;써브웨이 클럽/에다가 치즈는 /cheese;슈레드 치츠/로 해주시고, 채소는 /vegetable;오이/  빼주시고, /sauce;레드와인식초/ 로 해주세요.
/bread;플랫/ /length;30센치/ /sandwich;햄/에다가 치즈는 /cheese;슈레드/로 해주시고, 채소는 /vegetable;올리브/  빼주시고, /sauce;허니 머스타드/ 로 해주세요.
/bread;하티/ /length;15센치미터/ /sandwich;이탈리안 BLT/에다가 치즈는 /cheese;모차렐라/로 해주시고, 채소는 /vegetable;피클/  빼주시고, /sauce;머스타드/ 로 해주세요.
/bread;위트/ /length;15cm/ /sandwich;이탈리안 비엠티/에다가 치즈는 /cheese;슈레드/로 해주시고, 채소는 /vegetable;오이/  빼주시고, /sauce;올리브 오일/ 으로 해주세요.
/bread;파마산/ /length;15센티미터/ /sandwich;에그마요/에다가 치즈는 /cheese;아메리칸/으로 해주시고, 채소는 /vegetable;토마토/  빼주시고, /sauce;레드와인식초/ 로 해주세요.
/bread;파마산오레가노/ /length;15센티/ /sandwich;풀드 포크 바비큐/에다가 치즈는 /cheese;모차렐라 치츠/로 해주시고, 채소는 /vegetable;양상추/  빼주시고, /sauce;후추/ 로 해주세요.
/bread;화이트/ /length;15센치미터/ /sandwich;쉬림프/에다가 치즈는 /cheese;모짜렐라/로 해주시고, 채소는 /vegetable;피클/  빼주시고, /sauce;마요네즈/ 로 해주세요.
/bread;화이트/ /length;30

In [43]:
print(len(set(complex_data)))
with open('data.csv', 'a') as f:
    for sentence in complex_data:
        f.write(sentence+'\n')

200


In [44]:
## {길이} 없을 때
# {빵} {샌드위치}에다가 치즈는 {치즈}로 해주시고, 채소는 {채소} 빼주시고,
# 소스는 {소스}로 해주세요.
complex_front = []
for bread in menu['bread']:
    bread = '/bread;'+bread+'/ '
    for sandwich in menu['sandwich']:
        sandwich = '/sandwich;'+sandwich+'/'
        for cheese in menu['cheese']:
            cheese_predicate = '으로 해주시고, '
            if has_coda(cheese):
                cheese_predicate = '로 해주시고, '
            cheese = '/cheese;'+cheese+'/'
            complex_front.append(bread+sandwich+\
                                '에다가 치즈는 '+cheese+\
                               cheese_predicate)
MAX_VEG = 1
MAX_SAUCE = 1
complex_back = []
for i in range(1,MAX_VEG+1):
    for p_veg in permutations(menu['vegetable'],i):
        vegs = ''
        for veg in p_veg:
            vegs+='/vegetable;'+veg+'/ '
        for i in range(1,MAX_SAUCE+1):
            for p_sauce in permutations(menu['sauce'],i):
                sauces = ''
                for sauce in p_sauce:
                    sauces+='/sauce;'+sauce+'/ '
                sauce_predicate = '으로 해주세요.'
                if has_coda(sauce):
                    sauce_predicate = '로 해주세요.'
                complex_back.append('채소는 '+\
                                   vegs+' 빼주시고요, '+\
                                   sauces+sauce_predicate)
                        
'''
print(len(complex_front)) 
print(len(complex_back))     
print(len(complex_back)*len(complex_front))
for front in complex_front:
    for back in complex_back:
      complex_data.append(front+back)
'''
complex_data = []
for _ in range(200):
    complex_data.append(random.choice(complex_front)+random.choice(complex_back))

print(len(complex_data))
print(*complex_data[:25], sep='\n')   

200
/bread;파마산/ /sandwich;에그마요/에다가 치즈는 /cheese;모차렐라 치츠/로 해주시고, 채소는 /vegetable;피망/  빼주시고요, /sauce;랜치/ 로 해주세요.
/bread;위트/ /sandwich;쉬림프/에다가 치즈는 /cheese;슈레드 치츠/로 해주시고, 채소는 /vegetable;피클/  빼주시고요, /sauce;랜치/ 로 해주세요.
/bread;플랫/ /sandwich;페퍼로니 피자 썹/에다가 치즈는 /cheese;모짜렐라/로 해주시고, 채소는 /vegetable;토마토/  빼주시고요, /sauce;소금/ 으로 해주세요.
/bread;파마산/ /sandwich;베지/에다가 치즈는 /cheese;슈레드 치츠/로 해주시고, 채소는 /vegetable;피클/  빼주시고요, /sauce;스위트 어니언/ 으로 해주세요.
/bread;플랫/ /sandwich;페퍼로니 피자 썹/에다가 치즈는 /cheese;모짜렐라/로 해주시고, 채소는 /vegetable;양상추/  빼주시고요, /sauce;스모크 바비큐/ 로 해주세요.
/bread;허니오트/ /sandwich;참치/에다가 치즈는 /cheese;모짜렐라/로 해주시고, 채소는 /vegetable;피클/  빼주시고요, /sauce;스위트 칠리/ 로 해주세요.
/bread;화이트/ /sandwich;로티세리 바비큐 치킨/에다가 치즈는 /cheese;모차렐라 치츠/로 해주시고, 채소는 /vegetable;피클/  빼주시고요, /sauce;핫 칠리/ 로 해주세요.
/bread;위트/ /sandwich;베지/에다가 치즈는 /cheese;아메리칸 치즈/로 해주시고, 채소는 /vegetable;피망/  빼주시고요, /sauce;허니 머스타드/ 로 해주세요.
/bread;허니오트/ /sandwich;스테이크 앤 치즈/에다가 치즈는 /cheese;슈레드 치츠/로 해주시고, 채소는 /vegetable;올리브/  빼주시고요, /sauce;스위트 칠리/ 로 해주세요.
/bread;화이트/ /sandwic

In [45]:
print(len(set(complex_data)))
with open('data.csv', 'a') as f:
    for sentence in complex_data:
        f.write(sentence+'\n')

200


In [47]:
## {치즈} 없을 때
# {빵} {길이}에 {샌드위치}로 해주시고, 채소는 {채소} 빼주시고,
# 소스는 {소스}로 해주세요.
complex_front = []
for bread in menu['bread']:
    bread = '/bread;'+bread+'/ '
    for length in menu['length']:
        length = '/length;'+length+'/ '
        for sandwich in menu['sandwich']:
            sandwich = '/sandwich;'+sandwich+'/'
            predicate = '으로 해주시고요, '
            if has_coda(sandwich):
                predicate = '로 해주시고요, '
            complex_front.append(bread+length+sandwich+predicate)
     
MAX_VEG = 2
MAX_SAUCE = 1
complex_back = []
for i in range(1,MAX_VEG+1):
    for p_veg in permutations(menu['vegetable'],i):
        vegs = ''
        for veg in p_veg:
            vegs+='/vegetable;'+veg+'/ '
        for i in range(1,MAX_SAUCE+1):
            for p_sauce in permutations(menu['sauce'],i):
                sauces = ''
                for sauce in p_sauce:
                    sauces+='/sauce;'+sauce+'/ '
                sauce_predicate = '으로 해주세요.'
                if has_coda(sauce):
                    sauce_predicate = '로 해주세요.'
                complex_back.append('채소는 '+\
                                   vegs+' 빼주시고, '+\
                                   sauces+sauce_predicate)
          

'''
print(len(complex_front)) 
print(len(complex_back))     
print(len(complex_back)*len(complex_front))
for front in complex_front:
    for back in complex_back:
      complex_data.append(front+back)
'''

complex_data = []
for _ in range(200):
    complex_data.append(random.choice(complex_front)+random.choice(complex_back))

print(len(complex_data))
print(*complex_data[:25], sep='\n')   

200
/bread;파마산오레가노/ /length;15센치미터/ /sandwich;풀드 포크 바비큐/로 해주시고요, 채소는 /vegetable;양파/ /vegetable;할라피뇨/  빼주시고, /sauce;소금/ 으로 해주세요.
/bread;화이트/ /length;15센티/ /sandwich;로스트 치킨/로 해주시고요, 채소는 /vegetable;피클/ /vegetable;토마토/  빼주시고, /sauce;후추/ 로 해주세요.
/bread;위트/ /length;6인치/ /sandwich;페퍼 치킨 슈니첼/로 해주시고요, 채소는 /vegetable;올리브/ /vegetable;오이/  빼주시고, /sauce;올리브 오일/ 으로 해주세요.
/bread;파마산오레가노/ /length;15센치미터/ /sandwich;터키/로 해주시고요, 채소는 /vegetable;피망/ /vegetable;오이/  빼주시고, /sauce;허니 머스타드/ 로 해주세요.
/bread;하티/ /length;30센티/ /sandwich;터키 베이컨 아보카도/로 해주시고요, 채소는 /vegetable;올리브/ /vegetable;할라피뇨/  빼주시고, /sauce;마요네즈/ 로 해주세요.
/bread;위트/ /length;풋롱/ /sandwich;쉬림프/로 해주시고요, 채소는 /vegetable;피망/ /vegetable;오이/  빼주시고, /sauce;핫 칠리/ 로 해주세요.
/bread;플랫/ /length;15센티미터/ /sandwich;스테이크 앤 치즈/로 해주시고요, 채소는 /vegetable;피클/  빼주시고, /sauce;홀스래디쉬/ 로 해주세요.
/bread;파마산/ /length;30센티/ /sandwich;BMT/로 해주시고요, 채소는 /vegetable;오이/ /vegetable;피망/  빼주시고, /sauce;랜치/ 로 해주세요.
/bread;플랫/ /length;6인치/ /sandwich;로티세리 바비큐 치킨/로 해주시고요, 채소는 /vegetable;올리브/ /veg

In [48]:
print(len(set(complex_data)))
with open('data.csv', 'a') as f:
    for sentence in complex_data:
        f.write(sentence+'\n')

200


In [49]:
# {채소} 없을 때
# {소스} 없을 때

In [58]:
# 슬롯 없는 문장
no_slot_data = pd.read_table('no_slot_data.txt',sep='\n',header=None)
no_slot_data.head()

,0
0,잠시만요.
1,잠시만 기다려주세요.
2,잠깐만요.
3,안녕하세요.
4,안녕하십니까.


In [53]:
len(no_slot_data)

84

In [60]:
with open('data.csv', 'a') as f:
    for sentence in no_slot_data[0]:
        f.write(sentence+'\n')